In [ ]:
import joblib
import numpy as np
from http.server import BaseHTTPRequestHandler, HTTPServer

# Load the trained machine learning model
model = joblib.load('finalized_model.sav')

class MyHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            
            with open('index.html', 'rb') as file:
                self.wfile.write(file.read())
    
    def do_POST(self):
        if self.path == '/':
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length).decode('utf-8')
            
            # Extract and preprocess feature data from form submission
            gender = post_data.split('&')[0].split('=')[1]
            ssc_p = float(post_data.split('&')[1].split('=')[1])
            hsc_p = float(post_data.split('&')[2].split('=')[1])
            degree_p = float(post_data.split('&')[3].split('=')[1])
            workex = post_data.split('&')[4].split('=')[1]
            etest_p = float(post_data.split('&')[5].split('=')[1])
            specialisation = post_data.split('&')[6].split('=')[1]
            mba_p = float(post_data.split('&')[7].split('=')[1])
            Arts = post_data.split('&')[8].split('=')[1]
            Commerce = post_data.split('&')[9].split('=')[1]
            Science = post_data.split('&')[10].split('=')[1]
            Comm_Mgmt = post_data.split('&')[11].split('=')[1]
            Others = post_data.split('&')[12].split('=')[1]
            Sci_Tech = post_data.split('&')[13].split('=')[1]
           
            
            # Convert categorical features to binary values
            gender = 1 if gender == 'Male' else 0
            workex = 1 if workex == 'Yes' else 0
            specialisation = 1 if specialisation == 'Mkt&HR' else 0
            Arts = 1 if Arts == 'Yes' else 0
            Commerce = 1 if Commerce == 'Yes' else 0
            Science = 1 if Science == 'Yes' else 0
            Comm_Mgmt = 1 if Comm_Mgmt == 'Yes' else 0
            Others = 1 if Others == 'Yes' else 0
            Sci_Tech = 1 if Sci_Tech == 'Yes' else 0
            
            # Create input data for prediction
            input_data = np.array([[gender, ssc_p, hsc_p, degree_p, workex, etest_p, specialisation,
                                    mba_p, Arts, Commerce, Science, Comm_Mgmt, Others, Sci_Tech]])
            
            # Make a prediction using the model
            prediction = model.predict(input_data)
            
            # Format the prediction result
            result = "Placed" if prediction == 1 else "Not Placed"
            
            # Return the HTML response
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            response = f"<p>Prediction Result: {result}</p>".encode('utf-8')
            self.wfile.write(response)

# Create a simple HTTP server
with HTTPServer(("localhost", 8000), MyHandler) as httpd:
    print("Server started on port 8000")
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        httpd.server_close()
        print("Server stopped")


Server started on port 8000


127.0.0.1 - - [06/Aug/2023 17:40:15] "GET / HTTP/1.1" 200 -
C:\Users\Kalyani\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
127.0.0.1 - - [06/Aug/2023 17:40:27] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2023 17:44:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2023 17:45:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2023 17:46:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2023 17:46:43] "GET / HTTP/1.1" 200 -
C:\Users\Kalyani\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
127.0.0.1 - - [06/Aug/2023 17:46:57] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2023 17:47:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2023 17:48:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2023 17:49:16] "GET / HTTP/1.1" 200 -
C:\Users\Kalyani\anaconda3